# **Customer Satisfaction Prediction**

**Project Type:** ML + SQL + Excel  
**Tools Used:** Python, Pandas, DuckDB (SQL),  
**Author:** _Sanya Sharma_  
**Date:** 25 July 2025

**----------**

## **1. Project Objective**
To analyze customer support tickets and build a predictive model that estimates customer satisfaction based on ticket characteristics (type, channel, priority, etc.).


**-----------**

## **2. Dataset Overview**


- **Filename:** `customer_support_tickets.csv`
- **Total Records:** ~8,400
- **Features:**  
  - `TicketType`, `TicketChannel`, `TicketPriority`, `CustomerSatisfactionRating`, etc.
- **Goal Column:** `CustomerSatisfactionRating`

**----------**

## **3. Data Upload & Preparation**

### Upload the CSV file

In [1]:
from google.colab import files
uploaded = files.upload()


Saving customer_support_tickets.csv to customer_support_tickets.csv


### Load It Into a Pandas DataFrame

In [2]:
import pandas as pd

df = pd.read_csv("customer_support_tickets.csv")
df.shape  # Check how many rows/columns
df.head()

,TicketID,CustomerName,CustomerEmail,CustomerAge,CustomerGender,ProductPurchased,DateOfPurchase,TicketType,TicketSubject,TicketDescription,TicketStatus,Resolution,TicketPriority,TicketChannel,FirstResponseTime,TimetoResolution,CustomerSatisfactionRating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,22-03-2021,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,01-06-2023 12:15,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,22-05-2021,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,01-06-2023 16:45,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,14-07-2020,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,01-06-2023 11:14,01-06-2023 18:05,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,13-11-2020,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,01-06-2023 07:29,01-06-2023 01:57,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,04-02-2020,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,01-06-2023 00:12,01-06-2023 19:53,1.0


### Install DuckDB

In [3]:
!pip install duckdb --quiet
import duckdb


### Basic Info

In [4]:
duckdb.query("SELECT COUNT(*) FROM df").to_df()


,count_star()
0,8469


In [5]:
duckdb.query("SELECT * FROM df LIMIT 5").to_df()


,TicketID,CustomerName,CustomerEmail,CustomerAge,CustomerGender,ProductPurchased,DateOfPurchase,TicketType,TicketSubject,TicketDescription,TicketStatus,Resolution,TicketPriority,TicketChannel,FirstResponseTime,TimetoResolution,CustomerSatisfactionRating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,22-03-2021,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,None,Critical,Social media,01-06-2023 12:15,None,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,22-05-2021,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,None,Critical,Chat,01-06-2023 16:45,None,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,14-07-2020,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,01-06-2023 11:14,01-06-2023 18:05,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,13-11-2020,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,01-06-2023 07:29,01-06-2023 01:57,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,04-02-2020,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,01-06-2023 00:12,01-06-2023 19:53,1.0


### Count Missing Values Per Column

In [13]:
duckdb.query("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN CustomerName IS NULL THEN 1 ELSE 0 END) AS missing_customer_name,
  SUM(CASE WHEN CustomerEmail IS NULL THEN 1 ELSE 0 END) AS missing_customer_email,
  SUM(CASE WHEN CustomerAge IS NULL THEN 1 ELSE 0 END) AS missing_customer_age,
  SUM(CASE WHEN CustomerGender IS NULL THEN 1 ELSE 0 END) AS missing_customer_gender,
  SUM(CASE WHEN ProductPurchased IS NULL THEN 1 ELSE 0 END) AS missing_product,
  SUM(CASE WHEN DateOfPurchase IS NULL THEN 1 ELSE 0 END) AS missing_date,
  SUM(CASE WHEN TicketType IS NULL THEN 1 ELSE 0 END) AS missing_ticket_type,
  SUM(CASE WHEN TicketSubject IS NULL THEN 1 ELSE 0 END) AS missing_subject,
  SUM(CASE WHEN TicketDescription IS NULL THEN 1 ELSE 0 END) AS missing_description,
  SUM(CASE WHEN TicketStatus IS NULL THEN 1 ELSE 0 END) AS missing_status,
  SUM(CASE WHEN Resolution IS NULL THEN 1 ELSE 0 END) AS missing_resolution,
  SUM(CASE WHEN TicketPriority IS NULL THEN 1 ELSE 0 END) AS missing_priority,
  SUM(CASE WHEN TicketChannel IS NULL THEN 1 ELSE 0 END) AS missing_channel,
  SUM(CASE WHEN FirstResponseTime IS NULL THEN 1 ELSE 0 END) AS missing_first_response,
  SUM(CASE WHEN TimetoResolution IS NULL THEN 1 ELSE 0 END) AS missing_resolution_time,
  SUM(CASE WHEN CustomerSatisfactionRating IS NULL THEN 1 ELSE 0 END) AS missing_rating
FROM df
""").to_df()


,total_rows,missing_customer_name,missing_customer_email,missing_customer_age,missing_customer_gender,missing_product,missing_date,missing_ticket_type,missing_subject,missing_description,missing_status,missing_resolution,missing_priority,missing_channel,missing_first_response,missing_resolution_time,missing_rating
0,8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5700.0,0.0,0.0,2819.0,5700.0,5700.0


### Tickets by Type

In [14]:
duckdb.query("""
SELECT TicketType, COUNT(*) AS ticket_count
FROM df
GROUP BY TicketType
ORDER BY ticket_count DESC
""").to_df()


,TicketType,ticket_count
0,Refund request,1752
1,Technical issue,1747
2,Cancellation request,1695
3,Product inquiry,1641
4,Billing inquiry,1634


### Average satisfaction by Channel

In [15]:
duckdb.query("""
SELECT TicketChannel, AVG(CustomerSatisfactionRating) AS avg_rating
FROM df
GROUP BY TicketChannel
ORDER BY avg_rating DESC
""").to_df()


,TicketChannel,avg_rating
0,Chat,3.083086
1,Social media,2.969298
2,Email,2.963889
3,Phone,2.952243


### Gender-wise satisfaction

In [16]:
duckdb.query("""
SELECT CustomerGender, AVG(CustomerSatisfactionRating) AS avg_satisfaction
FROM df
GROUP BY CustomerGender
""").to_df()


,CustomerGender,avg_satisfaction
0,Female,2.971545
1,Male,3.028384
2,Other,2.974684


### Duplicate Ticket Check

In [17]:
duckdb.query("""
SELECT TicketID, COUNT(*) AS cnt
FROM df
GROUP BY TicketID
HAVING cnt > 1
""").to_df()


,TicketID,cnt


### Satisfaction by Product

In [18]:
duckdb.query("""
SELECT ProductPurchased, ROUND(AVG(CustomerSatisfactionRating), 2) AS avg_rating
FROM df
GROUP BY ProductPurchased
ORDER BY avg_rating DESC
""").to_df()


,ProductPurchased,avg_rating
0,Dyson Vacuum Cleaner,3.22
1,Google Pixel,3.22
2,Garmin Forerunner,3.21
3,Autodesk AutoCAD,3.20
4,Asus ROG,3.20
5,GoPro Hero,3.19
6,Nintendo Switch Pro Controller,3.18
7,Sony PlayStation,3.18
8,Microsoft Office,3.14
9,MacBook Pro,3.13


### Satisfaction by Priority

In [19]:
duckdb.query("""
SELECT TicketPriority, AVG(CustomerSatisfactionRating) AS avg_rating
FROM df
GROUP BY TicketPriority
ORDER BY avg_rating DESC
""").to_df()


,TicketPriority,avg_rating
0,Low,3.052795
1,High,2.982979
2,Medium,2.976945
3,Critical,2.958678


### Tickets by Age Groups

In [25]:
duckdb.query("""
SELECT
  CASE
    WHEN CustomerAge < 25 THEN 'Under 25'
    WHEN CustomerAge BETWEEN 25 AND 40 THEN '25-40'
    ELSE '40+'
  END AS age_group,
  COUNT(*) AS count,
  AVG(CustomerSatisfactionRating) AS avg_rating
FROM df
GROUP BY age_group
""").to_df()

,age_group,count,avg_rating
0,25-40,2494,3.032500
1,40+,4837,2.979592
2,Under 25,1138,2.951705


**----------**